In [8]:

pip install pytorch_fid

  Using cached pytorch_fid-0.3.0-py3-none-any.whl.metadata (5.3 kB)
Using cached pytorch_fid-0.3.0-py3-none-any.whl (15 kB)

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import save_image
import os
from pytorch_fid import fid_score

# Set your dataset path
DATASET_DIR = r"/Users/itsme_reddynaveen/Desktop/Shylendra_imagenet/sorted/"
REAL_IMAGES_PATH = "./real_images2/"
GENERATED_IMAGES_PATH = "./generated_images2/"
os.makedirs(REAL_IMAGES_PATH, exist_ok=True)
os.makedirs(GENERATED_IMAGES_PATH, exist_ok=True)

# Hyperparameters
latent_dim = 128
batch_size = 64
image_size = 64
num_epochs = 100
lr = 0.0002
beta1 = 0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Transformations
transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize to [-1, 1]
])

# Load dataset
dataset = datasets.ImageFolder(DATASET_DIR, transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# DCGAN Generator
class Generator(nn.Module):
    def __init__(self, latent_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.ConvTranspose2d(latent_dim, 512, kernel_size=4, stride=1, padding=0),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 3, kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        )

    def forward(self, z):
        return self.model(z)

# DCGAN Discriminator
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Flatten(),
            nn.Linear(512 * 4 * 4, 1),
            nn.Sigmoid()
        )

    def forward(self, img):
        return self.model(img)

# Initialize models
generator = Generator(latent_dim).to(device)
discriminator = Discriminator().to(device)

# Loss and optimizers
criterion = nn.BCELoss()
optimizer_G = optim.Adam(generator.parameters(), lr=lr, betas=(beta1, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=lr, betas=(beta1, 0.999))

# Training loop with try-except block
for epoch in range(num_epochs):
    try:
        for i, (imgs, _) in enumerate(dataloader):
            # Prepare real and fake labels
            real_imgs = imgs.to(device)
            real_labels = torch.ones(imgs.size(0), 1).to(device)
            fake_labels = torch.zeros(imgs.size(0), 1).to(device)

            # Train Discriminator
            optimizer_D.zero_grad()
            output_real = discriminator(real_imgs)
            real_loss = criterion(output_real, real_labels)
            z = torch.randn(imgs.size(0), latent_dim, 1, 1, device=device)
            fake_imgs = generator(z)
            output_fake = discriminator(fake_imgs.detach())
            fake_loss = criterion(output_fake, fake_labels)
            d_loss = real_loss + fake_loss
            d_loss.backward()
            optimizer_D.step()

            # Train Generator
            optimizer_G.zero_grad()
            output_fake = discriminator(fake_imgs)
            gen_loss = criterion(output_fake, real_labels)
            gen_loss.backward()
            optimizer_G.step()

            # Logging
            if i % 100 == 0:
                print(f"Epoch [{epoch+1}/{num_epochs}], Batch {i}/{len(dataloader)} "
                      f"D Loss: {d_loss.item():.4f}, G Loss: {gen_loss.item():.4f}")

        # Save generated samples
        with torch.no_grad():
            z = torch.randn(64, latent_dim, 1, 1, device=device)
            samples = generator(z)
            samples = (samples + 1) / 2  # Rescale to [0, 1]
            save_image(samples, f"{GENERATED_IMAGES_PATH}/epoch_{epoch+1}.png", nrow=8)
    except FileNotFoundError as e:
        print(f"File not found during training: {e}")
        continue  # Skip to the next epoch


Epoch [1/100], Batch 0/137 D Loss: 1.5993, G Loss: 3.0374
Epoch [1/100], Batch 100/137 D Loss: 0.2536, G Loss: 5.1556
Epoch [2/100], Batch 0/137 D Loss: 0.8376, G Loss: 2.6610
Epoch [2/100], Batch 100/137 D Loss: 0.2998, G Loss: 2.4278
Epoch [3/100], Batch 0/137 D Loss: 2.2511, G Loss: 2.3604
Epoch [3/100], Batch 100/137 D Loss: 0.4559, G Loss: 2.3719
Epoch [4/100], Batch 0/137 D Loss: 0.4341, G Loss: 4.9806
Epoch [4/100], Batch 100/137 D Loss: 1.2069, G Loss: 5.3924
Epoch [5/100], Batch 0/137 D Loss: 1.1710, G Loss: 4.3157
Epoch [5/100], Batch 100/137 D Loss: 0.7046, G Loss: 3.5932
Epoch [6/100], Batch 0/137 D Loss: 3.2423, G Loss: 7.1496
Epoch [6/100], Batch 100/137 D Loss: 0.3093, G Loss: 4.1285
Epoch [7/100], Batch 0/137 D Loss: 0.9963, G Loss: 7.6055
Epoch [7/100], Batch 100/137 D Loss: 0.7394, G Loss: 6.3426
Epoch [8/100], Batch 0/137 D Loss: 4.0496, G Loss: 5.9850
Epoch [8/100], Batch 100/137 D Loss: 0.5409, G Loss: 2.3289
Epoch [9/100], Batch 0/137 D Loss: 0.9548, G Loss: 1.770

Downloading: "https://github.com/mseitzer/pytorch-fid/releases/download/fid_weights/pt_inception-2015-12-05-6726825d.pth" to /Users/itsme_reddynaveen/.cache/torch/hub/checkpoints/pt_inception-2015-12-05-6726825d.pth
100%|██████████| 91.2M/91.2M [00:01<00:00, 49.2MB/s]


ValueError: batch_size should be a positive integer value, but got batch_size=0

In [ ]:

# Calculate FID score
fid_value = fid_score.calculate_fid_given_paths(
    paths=[REAL_IMAGES_PATH, GENERATED_IMAGES_PATH],
    batch_size=50,
    device=device,
    dims=2048,
)
print(f"FID Score: {fid_value}")
